# TFM: Predicción de la demanda energética - Parte 1: Carga de datos
#### Lara Palacios
## 0. Importación de librerías

In [ ]:
import pandas as pd
import json
from datetime import datetime

## 1. Carga de datos

### 1.1. Archivo cupsInfo

#### 1.1.1 Carga desde ficheros

In [ ]:
file1 = "zdatos_20230109/cupsInfo.json"

In [ ]:
with open(file1) as f:
    data1 = json.load(f)

In [ ]:
def flatten_cupsInfo(json_obj):
    flattened_data = []

    for cod_cliente, cliente_data in json_obj.items():
        #if cod_cliente in clientes: #para las pruebas
        cod_simel = cliente_data.get('cod_simel', None)
        tarifa = cliente_data.get('tarifa', None)
        cpostal = cliente_data.get('cpostal', None)
        municipio = cliente_data.get('municipio', None)
        provincia = cliente_data.get('provincia', None)
        comAutonoma = cliente_data.get('comAutonoma', None)
        autoconsumo = cliente_data.get('autoconsumo', None)
        potencia_generacion_kW = cliente_data.get('potencia_generacion_kW', None)
        esquema_medida = cliente_data.get('esquema_medida', None)
        subgrupo = cliente_data.get('subgrupo', None)
        alta_autoconsumo = cliente_data.get('alta_autoconsumo', None)
        baja_autoconsumo = cliente_data.get('baja_autoconsumo', None)
        potencia_contratada_kW = cliente_data.get('potencia_contratada_kW', {})
        p1 = potencia_contratada_kW.get('p1', None)
        p2 = potencia_contratada_kW.get('p2', None)
        p3 = potencia_contratada_kW.get('p3', None)
        p4 = potencia_contratada_kW.get('p4', None)
        p5 = potencia_contratada_kW.get('p5', None)
        p6 = potencia_contratada_kW.get('p6', None)

        flattened_data.append({
            'PK_CLIENTE': int(cod_cliente),
            'TARIFA': str(tarifa),
            'MUNICIPIO': str(municipio),
            'PROVINCIA': str(provincia),
            'KW_CONTRATADA_P1': float(p1),
            'KW_CONTRATADA_P2': float(p2),
            'KW_CONTRATADA_P3': float(p3),
            'KW_CONTRATADA_P4': float(p4),
            'KW_CONTRATADA_P5': float(p5),
            'KW_CONTRATADA_P6': float(p6),
            'AUTOCONSUMO': int(autoconsumo),
            'POTENCIA_GENERACION_KW': float(potencia_generacion_kW),
            'ESQUEMA_MEDIDA': str(esquema_medida),
            'SUBGRUPO': str(subgrupo),
            'ALTA_AUTOCONSUMO': datetime.strptime(alta_autoconsumo, '%Y-%m-%d'),
            'BAJA_AUTOCONSUMO': datetime.strptime(baja_autoconsumo, '%Y-%m-%d')
            })

    return flattened_data

In [ ]:
datos1 = data1.get("datos", {})
flattened_data1 = flatten_cupsInfo(datos1)
df1 = pd.DataFrame(flattened_data1)

In [ ]:
df1.shape

(11786, 16)

In [ ]:
df1.isnull().sum()

PK_CLIENTE                0
TARIFA                    0
MUNICIPIO                 0
PROVINCIA                 0
KW_CONTRATADA_P1          0
KW_CONTRATADA_P2          0
KW_CONTRATADA_P3          0
KW_CONTRATADA_P4          0
KW_CONTRATADA_P5          0
KW_CONTRATADA_P6          0
AUTOCONSUMO               0
POTENCIA_GENERACION_KW    0
ESQUEMA_MEDIDA            0
SUBGRUPO                  0
ALTA_AUTOCONSUMO          0
BAJA_AUTOCONSUMO          0
dtype: int64

In [ ]:
df1.to_csv('cupsInfo.csv', index = False)

#### 1.1.2 Carga desde .csv

In [ ]:
df1 = pd.read_csv('cupsInfo.csv')
df1.shape

(11786, 16)

### 1.2. Archivos cups

#### 1.2.1 Carga desde ficheros

In [ ]:
import os
cwd = os.getcwd()
dir2 = cwd + "\\datos_20230109\\cups"
files = [dir2 + "\\" + file for file in os.listdir(dir2)]
files[0]

'C:\\Users\\larap\\OneDrive - Universidad de La Rioja\\TFM - Lara\\datos_20230109\\cups\\1.json'

In [ ]:
len(files)

11786

In [ ]:
def flatten_cups(json_obj):
    flattened_data = []

    for cod_cliente, cliente_data in json_obj.items():

            for fecha, fecha_data in cliente_data.items():
                for hora, hora_data in fecha_data.items():
                    entry = hora_data.get('entrada', None)
                    exit = hora_data.get('salida', None)

                    flattened_data.append({
                        'PK_CLIENTE': int(cod_cliente),
                        'PK_ANYOMESDIA': datetime.strptime(fecha, '%Y-%m-%d'),
                        'PK_HORA': int((int(hora)/100)-1),
                        'ENTRADA': entry,
                        'SALIDA': exit
                    })

    return flattened_data

In [ ]:
df2 = pd.DataFrame()
i=0

for file2 in files:
    # Lee el archivo JSON
    with open(file2) as f:
        data2 = json.load(f)

    datos2 = data2.get("datos", {})

    # Aplana el JSON
    flattened_data2 = flatten_cups(datos2)
    df_concat2 = pd.DataFrame(flattened_data2)

    #concatenamos todos los dataframes
    df2 = pd.concat([df2, df_concat2], axis=0)

    # i = i+1
    # if i%500 == 0:
    #     print(f"{str(i)} archivos leídos.")

In [ ]:
df2.isna().sum()

PK_CLIENTE             0
PK_ANYOMESDIA          0
PK_HORA                0
ENTRADA          1612265
SALIDA           1612288
dtype: int64

In [ ]:
df2['PK_ANYOMESDIA'].describe()

C:\Users\larap\AppData\Local\Temp\ipykernel_22976\1366616956.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_2['PK_ANYOMESDIA'].describe()


count                17396856
unique                     62
top       2023-01-03 00:00:00
freq                   282816
first     2022-11-09 00:00:00
last      2023-01-09 00:00:00
Name: PK_ANYOMESDIA, dtype: object

In [ ]:
df2.to_parquet('cups.parquet')

#### 1.2.2 Carga desde .parquet

In [ ]:
df2 = pd.read_parquet('cups.parquet')
df2.shape

(17396856, 5)

In [ ]:
df2 = df2.dropna().reset_index(drop = True)
df2.shape

(15784568, 5)

### 1.3. Archivos tiempo

#### 1.3.1 Carga desde ficheros

In [ ]:
import os
cwd = os.getcwd()
dir3 = cwd + "\\datos_20230109\\tiempo"
clientes = [file for file in os.listdir(dir3)]
len(clientes)

11786

In [ ]:
def flatten_tiempo(json_obj):
    flattened_data = []

    municipio = json_obj['municipio']
    fecha = datetime.strptime(json_obj['fecha'], '%Y-%m-%d')
    orto = datetime.strptime(json_obj['orto'], '%Y-%m-%d %H:%M:%S')
    ocaso = datetime.strptime(json_obj['ocaso'], '%Y-%m-%d %H:%M:%S')

    for hora, hora_data in json_obj.items():
        if hora.isdigit():
            nieve = hora_data.get('nieve', None)
            estado_cielo = hora_data.get('estadoCielo', None)
            descripcion = hora_data.get('descripcion', None)
            sens_termica = hora_data.get('sensTermica', None)
            temperatura = hora_data.get('temperatura', None)
            precipitacion = hora_data.get('precipitacion', None)
            humedad_relativa = hora_data.get('humedadRelativa', None)
            viento_direccion = hora_data.get('vientoDireccion', None)
            viento_velocidad = hora_data.get('vientoVelocidad', None)
            viento_and_racha_max = hora_data.get('vientoAndRachaMax', None)

            flattened_data.append({
                'PK_ANYOMESDIA': fecha,
                'PK_HORA': int(hora),
                'MUNICIPIO': str(municipio),
                'ORTO' : str(orto)[11:16],
                'OCASO': str(ocaso)[11:16],
                'NIEVE': nieve,
                'ESTADO_CIELO': estado_cielo,
                'DESCRIPCION': descripcion,
                'SENS_TERMICA': int(sens_termica),
                'TEMPERATURA': int(temperatura),
                'PRECIPITACION': precipitacion,
                'HUMEDAD_RELATIVA': humedad_relativa,
                'VIENTO_DIRECCION': viento_direccion,
                'VIENTO_VELOCIDAD': viento_velocidad,
                'VIENTO_AND_RACHA_MAX': viento_and_racha_max
            })

    return flattened_data

In [ ]:
df3 = pd.DataFrame()
i = 0
j = 0

for cliente3 in clientes:
    dir3_2 = dir3 + "\\" + cliente3
    files3 = [dir3_2 + "\\" + file for file in os.listdir(dir3_2)]
    print('El cliente ' + str(cliente3) + ' tiene ' + str(len(files3)) + ' archivos.')
    for file3 in files3:
        # Lee el archivo JSON
        with open(file3) as f:
            data3 = json.load(f)

        datos3 = data3.get("datos", {})

        try:
            # Aplana el JSON
            flattened_data3 = flatten_tiempo(datos3)
            df3_aux = pd.DataFrame(flattened_data3)
            df3_aux.insert(0, 'PK_CLIENTE', int(cliente3))

            #concatenamos todos los dataframes
            df3 = pd.concat([df3, df3_aux], axis=0)
        except TypeError as e:
            print('Ha fallado el fichero ' + str(file3) + ' del cliente ' + str(cliente3) + ' con el TypeError ' + str(e))

    # i = i +1
    # if i%500 == 0:
    #     print(f"{str(i)} clientes leídos.")

In [ ]:
df3.shape

(1687139, 16)

In [ ]:
df_3 = df3

In [ ]:
df_3.to_parquet('tiempo.parquet')

#### 1.3.2 Carga desde .parquet

In [ ]:
df_3 = pd.read_parquet('tiempo.parquet')
df_3.shape

(15397206, 16)